Render

In [7]:
import importlib
import os
from tkinter import NO
import yaml
import argparse
from datetime import datetime

from sacd.env import make_pytorch_env
from sacd.agent import SacdAgent, SharedSacdAgent


import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT
from datetime import datetime
import cv2
from tqdm import tqdm
import numpy as np
from datetime import datetime
from gym.wrappers import GrayScaleObservation
from smb_env_fct import renderEnv, plotRewardandTime, DeadlockEnv, SkipFrame, Downsample, GrayScale, move_state_channels, MarioDeathLoggerWrapper


In [8]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--config', type=str, default=os.path.join('config', 'sacd.yaml'))
parser.add_argument('--shared', action='store_true')
parser.add_argument('--env_id', type=str, default='SuperMarioBros-v2')
parser.add_argument('--cuda', action='store_true')
parser.add_argument('--seed', type=int, default=0)
args, unknown = parser.parse_known_args()

with open(args.config) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [9]:
frameskip = 4
load_model_path = "logs/SuperMarioBros-v2/sacd-seed0-20240704-0106/model/final"

In [10]:
env = gym_super_mario_bros.make(args.env_id, apply_api_compatibility=True)  #the environment. v0 is with original background, v1 has the background removed
env = JoypadSpace(env, SIMPLE_MOVEMENT)               #The Joypadspace sets the available actions. We use SIMPLE_MOVEMENT.
env = SkipFrame(env, skip=frameskip)                  #Skipframewrapper to skip some frames
env = DeadlockEnv(env,threshold=10)                   #Deadlock environment wrapper to stop the game if mario is stuck at a pipe
env = MarioDeathLoggerWrapper(env, logfile=logfile, env_id=None, select_random_stage=True)

In [11]:
# Specify the directory to log.
name = args.config.split('/')[-1].rstrip('.yaml')
if args.shared:
    name = 'shared-' + name
time = datetime.now().strftime("%Y%m%d-%H%M")
log_dir = os.path.join(
    'logs', args.env_id, f'{name}-seed{args.seed}-{time}')

# Create the agent.
Agent = SacdAgent if not args.shared else SharedSacdAgent
agent = Agent(
    env=env, test_env=env, log_dir=log_dir, cuda=args.cuda,
    seed=args.seed, **config)

In [12]:
# agent.evaluate()
agent.load_models(load_model_path)
agent.testnshow()

episode: 1 episode return: 158.75
episode: 2 episode return: 158.75
episode: 3 episode return: 158.75
episode: 4 episode return: 158.75
episode: 5 episode return: 158.75
episode: 6 episode return: 158.75
episode: 7 episode return: 158.75
episode: 8 episode return: 158.75
episode: 9 episode return: 158.75
episode: 10 episode return: 158.75
episode: 11 episode return: 158.75
episode: 12 episode return: 158.75
episode: 13 episode return: 158.75
episode: 14 episode return: 158.75


KeyboardInterrupt: 